<a href="https://colab.research.google.com/github/arezyhs/Python-Digital-Image-Processing-Projects/blob/main/digital-image-processing-project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 23051204335 && 23051204322
# Proses ini terdiri dari beberapa langkah berikut:
# 1. Deteksi Batas Dokumen: Menggunakan YOLOv8 untuk mendeteksi kontur dokumen, yang berfungsi sebagai langkah pertama.
# 2. Penciptaan Grid 2D Topologis: Menggunakan interpolasi polinomial kubik untuk mengoreksi distorsi dan menciptakan grid 2D yang meratakan dokumen.
# 3. Perbaikan Geometri Gambar: Remapping gambar asli berdasarkan grid interpolasi untuk memperoleh bentuk yang lebih mendekati dokumen aslinya.
# 4. Evaluasi OCR dan Pemulihan Geometri, menggunakan metrik OCR:
#       - Levenshtein distance,
#       - Jaro-Winkler similarity,
#       - Character Error Rate (CER)
#       - SSIM, MSE, dan NRMSE untuk mengukur kualitas pemulihan geometri dokumen.

In [ ]:
# Edge Detection

In [ ]:
# Grid

In [ ]:
# Geometry Fixer

In [ ]:
# Evaluasi OCR dan Geometry